In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader


In [17]:
import numpy as np
import torch

def projectile_trajectory_2d(m, dt, theta, v0, g = 9.81, x0 = 0, y0 = 0, N=1, dim=2):
    """
    Calculates a 2D cartesian trajectory of a projectile of mass m launched at angle theta with initial velocity v0.  
    
        Parameters:
    theta (int or float): angle between projectile trajectory and x-axis at t=0s in degrees \\
    v0 (int or float): the intial velocity in m/s \\
    m (int or float): mass of the projectile in kg \\
    dt (int or float): time step for simulation in s \\
    g (int or float): gravity constant (standard is 9.81) in m/s^2 \\
    x0, y0 (int or float): initial position of ball in m \\

        Returns: 
    The output is a numpy array containing the (x,y,t) coordinates of the projectile
    """

    # Convert theta to radians
    theta_rad = np.radians(theta)

    # Initial velocity components
    vx0 = v0 * np.cos(theta_rad)
    vy0 = v0 * np.sin(theta_rad)

    # Time array
    t_max = (vy0 + np.sqrt(vy0**2+2*g*y0)) / g
    t = np.arange(0, t_max, dt)
    T = len(t)

    # Compute trajectory
    x = x0 + vx0 * t
    y = y0 + vy0 * t - 0.5 * g * t**2

    # Remove negative y values and make np array
    valid = y >= 0
    x, y, t = x[valid], y[valid], t[valid]
    trajectory = np.column_stack((x,y))

    vx = np.full_like(x, vx0)
    vy = vy0 - g * t
    trajectory_velocities = np.column_stack((vx,vy))
    
    trajectory_data = {
        "time": torch.arange(T, dtype=torch.float32),
        "positions": torch.zeros((T, N, dim), dtype=torch.float32),
        "velocities": torch.zeros((T, N, dim), dtype=torch.float32),
        "masses": torch.arange(N, dtype=torch.float32)
    }
    
    trajectory_data["time"] = torch.tensor(t)
    trajectory_data["positions"] = torch.tensor(trajectory[:, :].reshape(T, N, dim))
    trajectory_data["velocities"] = torch.tensor(trajectory_velocities[:, :].reshape(T, N, dim))
    trajectory_data["masses"] = torch.tensor(m)

    return trajectory_data

In [18]:
traj_freefall = projectile_trajectory_2d(3, 0.2, 30, 50)

In [31]:
print(traj_freefall["masses"].numel())

1


In [9]:
data = Data(x=trajectory_data_freefall["masses"], edge_index=[0,0], pos=trajectory_data_freefall["positions"], time=trajectory_data_freefall["time"])
print(data)

Data(x=2, edge_index=[2], pos=[143, 1, 1], time=[143])


In [33]:
traj_freefall["time"][1].repeat(3,1)

tensor([[0.2000],
        [0.2000],
        [0.2000]], dtype=torch.float64)

In [ ]:
def node_data_list(trajectory_dict):
    data_list = []

    N = trajectory_dict["masses"].numel()
    
    for i in range(len(trajectory_dict["time"])):
        time_feature = trajectory_dict["time"][i].repeat(N,1)
        position_features = trajectory_dict["positions"][i]
        x_features = torch.cat([
            trajectory_dict["positions"][i],    # (N,dim)
            trajectory_dict["velocities"][i],   # (N, dim)
            trajectory_dict["masses"][i].unsqueeze(1),       # (N, 1)
            time_feature    # (N, 1)
        ], dim=1)   # tensor of shape (N, 2 + 2 + 1 + 1) + (N, 6) if dim = 2
        
        y_target = torch.tensor()

        data =Data(pos=position_features, time=time_feature)